In [48]:
import pandas as pd
import numpy as np
from utilities.reader_tools import GenericDatasetReader

df_carSales = GenericDatasetReader.DataframeFromCSV("Car_sales.csv")
df_carSales.head()

,Manufacturer,Model,Sales_in_thousands,__year_resale_value,Vehicle_type,Price_in_thousands,Engine_size,Horsepower,Wheelbase,Width,Length,Curb_weight,Fuel_capacity,Fuel_efficiency,Latest_Launch,Power_perf_factor
0,Acura,Integra,16.919,16.360,Passenger,21.50,1.8,140.0,101.2,67.3,172.4,2.639,13.2,28.0,2/2/2012,58.280150
1,Acura,TL,39.384,19.875,Passenger,28.40,3.2,225.0,108.1,70.3,192.9,3.517,17.2,25.0,6/3/2011,91.370778
2,Acura,CL,14.114,18.225,Passenger,NaN,3.2,225.0,106.9,70.6,192.0,3.470,17.2,26.0,1/4/2012,NaN
3,Acura,RL,8.588,29.725,Passenger,42.00,3.5,210.0,114.6,71.4,196.6,3.850,18.0,22.0,3/10/2011,91.389779
4,Audi,A4,20.397,22.255,Passenger,23.99,1.8,150.0,102.6,68.2,178.0,2.998,16.4,27.0,10/8/2011,62.777639


In [49]:
df_carSales["Vehicle_type"].value_counts()

Vehicle_type
Passenger    116
Car           41
Name: count, dtype: int64

In [50]:
df_carSales = df_carSales[["Manufacturer", "Sales_in_thousands", "Vehicle_type", "Price_in_thousands", "Engine_size", "Horsepower", "Fuel_capacity"]]
df_carSales

,Manufacturer,Sales_in_thousands,Vehicle_type,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
0,Acura,16.919,Passenger,21.50,1.8,140.0,13.2
1,Acura,39.384,Passenger,28.40,3.2,225.0,17.2
2,Acura,14.114,Passenger,NaN,3.2,225.0,17.2
3,Acura,8.588,Passenger,42.00,3.5,210.0,18.0
4,Audi,20.397,Passenger,23.99,1.8,150.0,16.4
...,...,...,...,...,...,...,...
152,Volvo,3.545,Passenger,24.40,1.9,160.0,15.8
153,Volvo,15.245,Passenger,27.50,2.4,168.0,17.9
154,Volvo,17.531,Passenger,28.80,2.4,168.0,17.9
155,Volvo,3.493,Passenger,45.50,2.3,236.0,18.5


In [51]:
df_carSales.nunique()

Manufacturer           30
Sales_in_thousands    157
Vehicle_type            2
Price_in_thousands    152
Engine_size            31
Horsepower             66
Fuel_capacity          55
dtype: int64

##### AGGREGATIONS -> agg()

In [52]:
# Sum of columns
df_carSales.agg("sum")

Manufacturer          AcuraAcuraAcuraAcuraAudiAudiAudiBMWBMWBMWBuick...
Sales_in_thousands                                             8320.698
Vehicle_type          PassengerPassengerPassengerPassengerPassengerP...
Price_in_thousands                                             4245.567
Engine_size                                                       477.5
Horsepower                                                      29008.0
Fuel_capacity                                                    2800.5
dtype: object

In [53]:
# It doesn't work with string values. Select only numerical one
df_carSales.select_dtypes(include=np.number).agg("mean")

Sales_in_thousands     52.998076
Price_in_thousands     27.390755
Engine_size             3.060897
Horsepower            185.948718
Fuel_capacity          17.951923
dtype: float64

In [54]:
# I can include any function in the list as arguments
df_carSales.select_dtypes(include=np.number).agg(["mean", "count"])

,Sales_in_thousands,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
mean,52.998076,27.390755,3.060897,185.948718,17.951923
count,157.000000,155.000000,156.000000,156.000000,156.000000


In [55]:
# Different aggregations over columns
df_carSales.agg({"Sales_in_thousands":["sum", "mean"],
                "Price_in_thousands":["sum", "max"]})

,Sales_in_thousands,Price_in_thousands
sum,8320.698000,4245.567
mean,52.998076,NaN
max,NaN,85.500


In [56]:
# In this case no sense.
df_carSales[["Sales_in_thousands", "Price_in_thousands"]].agg("sum", axis=1)

0      38.419
1      67.784
2      14.114
3      50.588
4      44.387
        ...  
152    27.945
153    42.745
154    46.331
155    48.993
156    54.969
Length: 157, dtype: float64

In [57]:
# Different from the previous approach, the Nan value here is present. If you use agg it will count Nan as 0
df_carSales["Sales_in_thousands"] + df_carSales["Price_in_thousands"]

0      38.419
1      67.784
2         NaN
3      50.588
4      44.387
        ...  
152    27.945
153    42.745
154    46.331
155    48.993
156    54.969
Length: 157, dtype: float64

In [58]:
# Rename indexes
df_carSales.agg(sum = ("Sales_in_thousands", "sum"),
                 y = ("Price_in_thousands", "sum"))

,Sales_in_thousands,Price_in_thousands
sum,8320.698,NaN
y,NaN,4245.567


##### SPLIT-APPLY-COMBINE STRATEGY -> BEHIND GROUP BY METHOD

In [59]:
df_carSales["Vehicle_type"].value_counts()

Vehicle_type
Passenger    116
Car           41
Name: count, dtype: int64

In [60]:
# Split data
passenger_filter = df_carSales["Vehicle_type"] == "Passenger"
car_filter = df_carSales["Vehicle_type"] == "Car"

In [61]:
# Apply an operation
passenger_sales_mean = df_carSales[passenger_filter]["Sales_in_thousands"].mean()
car_sales_mean = df_carSales[car_filter]["Sales_in_thousands"].mean()

In [62]:
# Combine results. I create a new DF with the data matched correctly
pd.DataFrame({"Vehicle_type":["Car", "Passenger"],
              "Sales_in_thousands": [car_sales_mean, passenger_sales_mean]})

,Vehicle_type,Sales_in_thousands
0,Car,80.622293
1,Passenger,43.234345


##### GROUP BY

If you apply an agg function on a df that contains string values, you will get an error.
If you group by per a certain string value and then aggregate, the other string values present in the df don't have to be considered.

To afford the problem:

1. Set numeric only parameter to true -> you don't have the full control of what is going on

2. Other solution:

import numpy as np

Step 1: Select only numeric columns and the grouping column
We use select_dtypes to grab all float/int columns automatically
numeric_cols = df_carSales.select_dtypes(include=[np.number]).columns.tolist()
df_subset = df_carSales[["Vehicle_type"] + numeric_cols]

Step 2: Perform the groupby
Now there is no risk of TypeError as all other columns are numeric
df_subset.groupby("Vehicle_type").mean()

In [63]:
df_carSales.groupby("Vehicle_type").mean(numeric_only=True)

# I can also add as_index=False if I want default indexing

,Sales_in_thousands,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
Vehicle_type,,,,,
Car,80.622293,26.319975,3.520000,186.400000,21.895000
Passenger,43.234345,27.763200,2.902586,185.793103,16.592241


In [64]:
groupedby_Manuf = df_carSales.groupby("Manufacturer")

In [65]:
# Type is not DataFrame, but dfgroupby (so with other characteristics)
type(groupedby_Manuf)

pandas.core.groupby.generic.DataFrameGroupBy

In [66]:
groupedby_Manuf.groups.keys()

dict_keys(['Acura', 'Audi', 'BMW', 'Buick', 'Cadillac', 'Chevrolet', 'Chrysler', 'Dodge', 'Ford', 'Honda', 'Hyundai', 'Infiniti', 'Jaguar', 'Jeep', 'Lexus', 'Lincoln', 'Mercedes-B', 'Mercury', 'Mitsubishi', 'Nissan', 'Oldsmobile', 'Plymouth', 'Pontiac', 'Porsche', 'Saab', 'Saturn', 'Subaru', 'Toyota', 'Volkswagen', 'Volvo'])

In [67]:
groupedby_Manuf.get_group("Ford")

,Manufacturer,Sales_in_thousands,Vehicle_type,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
46,Ford,70.227,Passenger,12.070,2.0,110.0,12.7
47,Ford,113.369,Passenger,21.560,3.8,190.0,15.7
48,Ford,35.068,Passenger,17.035,2.5,170.0,15.0
49,Ford,245.815,Passenger,17.885,3.0,155.0,16.0
50,Ford,175.670,Passenger,12.315,2.0,107.0,13.2
51,Ford,63.403,Passenger,22.195,4.6,200.0,19.0
52,Ford,276.747,Car,31.930,4.0,210.0,21.0
53,Ford,155.787,Car,21.410,3.0,150.0,26.0
54,Ford,125.338,Car,36.135,4.6,240.0,26.0
55,Ford,220.650,Car,12.050,2.5,119.0,20.0


In [68]:
groupedby_Manuf.mean(numeric_only=True)

,Sales_in_thousands,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
Manufacturer,,,,,
Acura,19.751250,30.633333,2.925000,200.000000,16.400000
Audi,13.519000,39.980000,2.933333,220.000000,19.533333
BMW,15.501667,33.096667,2.700000,185.333333,17.233333
Buick,60.504750,26.781250,3.625000,206.250000,17.750000
Cadillac,22.435600,40.254000,4.500000,256.000000,20.800000
Chevrolet,61.596111,20.022778,3.055556,171.111111,15.477778
Chrysler,28.817286,23.430833,2.783333,194.833333,16.483333
Dodge,82.740818,24.213636,3.709091,199.545455,21.581818
Ford,183.875909,21.047273,3.327273,170.090909,19.063636


In [69]:
groupedby_Manuf.sum(numeric_only=True)

,Sales_in_thousands,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
Manufacturer,,,,,
Acura,79.005,91.900,11.7,800.0,65.6
Audi,40.557,119.940,8.8,660.0,58.6
BMW,46.505,99.290,8.1,556.0,51.7
Buick,242.019,107.125,14.5,825.0,71.0
Cadillac,112.178,201.270,22.5,1280.0,104.0
Chevrolet,554.365,180.205,27.5,1540.0,139.3
Chrysler,201.721,140.585,16.7,1169.0,98.9
Dodge,910.149,266.350,40.8,2195.0,237.4
Ford,2022.635,231.520,36.6,1871.0,209.7


In [70]:
df_carSales.groupby("Vehicle_type").count()

# Some columns are NaN

,Manufacturer,Sales_in_thousands,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
Vehicle_type,,,,,,
Car,41,41,40,40,40,40
Passenger,116,116,115,116,116,116


In [71]:
df_carSales.isnull().sum()

Manufacturer          0
Sales_in_thousands    0
Vehicle_type          0
Price_in_thousands    2
Engine_size           1
Horsepower            1
Fuel_capacity         1
dtype: int64

In [72]:
# I also count the null value
df_carSales.groupby("Engine_size", dropna=False).count()

,Manufacturer,Sales_in_thousands,Vehicle_type,Price_in_thousands,Horsepower,Fuel_capacity
Engine_size,,,,,,
1.0,1,1,1,1,1,1
1.5,1,1,1,1,1,1
1.6,1,1,1,1,1,1
1.8,8,8,8,8,8,8
1.9,5,5,5,5,5,5
2.0,17,17,17,17,17,17
2.2,4,4,4,4,4,4
2.3,6,6,6,6,6,6
2.4,11,11,11,11,11,11


##### GROUPBY AND AGG()

In [73]:
# Find min and max value on each column

df_carSales.groupby("Vehicle_type").agg(["min", "max"])

Manufacturer         Sales_in_thousands           \
                      min     max                min      max   
Vehicle_type                                                    
Car              Cadillac  Toyota              9.126  540.561   
Passenger           Acura   Volvo              0.110  247.994   

             Price_in_thousands         Engine_size      Horsepower         \
                            min     max         min  max        min    max   
Vehicle_type                                                                 
Car                      11.528  60.105         2.0  5.7      119.0  300.0   
Passenger                 9.235  85.500         1.0  8.0       55.0  450.0   

             Fuel_capacity        
                       min   max  
Vehicle_type                      
Car                   15.1  32.0  
Passenger             10.3  23.7

In [74]:
df_carSales.groupby("Vehicle_type").agg(min_engine = ("Engine_size", "min"),
                                        max_horse=("Horsepower", "max"))

,min_engine,max_horse
Vehicle_type,,
Car,2.0,300.0
Passenger,1.0,450.0


##### What's behind the next instruction?

df_carSales.groupby("Manufacturer").sum()["Sales_in_thousands"]
df_carSales.groupby("Manufacturer").mean()["Price_in_thousands"]

Of course, the next instruction version is more scalable and powerful

In [75]:
# Sum of sales and mean of price for manufacterer
df_carSales.groupby("Manufacturer").agg(sumSales = ("Sales_in_thousands", "sum"),
                                        meanPrice=("Price_in_thousands", "mean"))

,sumSales,meanPrice
Manufacturer,,
Acura,79.005,30.633333
Audi,40.557,39.980000
BMW,46.505,33.096667
Buick,242.019,26.781250
Cadillac,112.178,40.254000
Chevrolet,554.365,20.022778
Chrysler,201.721,23.430833
Dodge,910.149,24.213636
Ford,2022.635,21.047273


##### APPLY MY FUNCTION: LAMBDA

In [76]:
# Example: apply the real value (so multiply by 1000)

df_carSales.groupby("Manufacturer").sum().apply(lambda x:x*1000)[["Sales_in_thousands", "Price_in_thousands"]]

,Sales_in_thousands,Price_in_thousands
Manufacturer,,
Acura,79005.0,91900.0
Audi,40557.0,119940.0
BMW,46505.0,99290.0
Buick,242019.0,107125.0
Cadillac,112178.0,201270.0
Chevrolet,554365.0,180205.0
Chrysler,201721.0,140585.0
Dodge,910149.0,266350.0
Ford,2022635.0,231520.0


In [ ]:
# Example: subtract the mean. Mean is calculated just on the numerical values, but the subtraction is made on all the columns (also the string and text ones). This behavior will cause problems in the future. Use transform instead

df_carSales.groupby("Manufacturer").apply(lambda x:x-x.mean(numeric_only=True)) #[["Sales_in_thousands", "Price_in_thousands"]]

TypeError: Could not convert string 'PassengerPassengerPassengerPassenger' to numeric

In [81]:
# Note that the argument of groupby() is used as the index of the generated df
df_carSales.groupby("Manufacturer").mean(numeric_only=True)

,Sales_in_thousands,Price_in_thousands,Engine_size,Horsepower,Fuel_capacity
Manufacturer,,,,,
Acura,19.751250,30.633333,2.925000,200.000000,16.400000
Audi,13.519000,39.980000,2.933333,220.000000,19.533333
BMW,15.501667,33.096667,2.700000,185.333333,17.233333
Buick,60.504750,26.781250,3.625000,206.250000,17.750000
Cadillac,22.435600,40.254000,4.500000,256.000000,20.800000
Chevrolet,61.596111,20.022778,3.055556,171.111111,15.477778
Chrysler,28.817286,23.430833,2.783333,194.833333,16.483333
Dodge,82.740818,24.213636,3.709091,199.545455,21.581818
Ford,183.875909,21.047273,3.327273,170.090909,19.063636


##### FILTERING AND AGGREGATION

In [86]:
df_carSales.mean(numeric_only=True)

Sales_in_thousands     52.998076
Price_in_thousands     27.390755
Engine_size             3.060897
Horsepower            185.948718
Fuel_capacity          17.951923
dtype: float64

In [109]:
def filtering_func(x):
    return x["Sales_in_thousands"].sum() > 52

filtered_df = df_carSales.groupby(["Manufacturer"]).filter(filtering_func)

In [ ]:
# Filtered out values
df_carSales.groupby(["Manufacturer"]).sum()["Sales_in_thousands"].sort_values().head(6)

Manufacturer
Porsche     12.128
Jaguar      15.467
Saab        21.306
Infiniti    23.713
Audi        40.557
BMW         46.505
Name: Sales_in_thousands, dtype: float64

In [ ]:
# Other method to detect filtered out values

filtered_list = set(filtered_df["Manufacturer"].unique())
original_list = set(df_carSales["Manufacturer"].unique())

original_list - filtered_list

In [ ]:
# Other method again

all_m_idx = pd.Index(df_carSales["Manufacturer"].unique())
filt_m_idx = pd.Index(filtered_df["Manufacturer"].unique())

excluded = all_m_idx.difference(filt_m_idx)
print(excluded.tolist())

['Audi', 'BMW', 'Infiniti', 'Jaguar', 'Porsche', 'Saab']
